<a href="https://colab.research.google.com/github/mohsen-goodarzi/DeepSpeech-with-keras/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras import layers
from keras import models
from keras import initializers
from keras import activations

Create DeepSpeech 1 model with dropout:

In [0]:

def DeepSpeech1(input_dim=26,output_dim=37,fc_size=1024, rnn_size=512,dropout=[0.1, 0.1, 0.1,0.1,0.1]):
  init=initializers.random_normal(stddev=0.046875)
  cliped_relu=activations.relu(max_value=20)
  model=models.Sequential()
  
  #first 3 FC layers
  model.add(layers.TimeDistributed(layers.Dense(fc_size,name='fc1',kernel_initializer=init,bias_initializer=init,activation=cliped_relu),input_shape=(None,input_dim)))
  model.add(layers.TimeDistributed(layers.Dropout(dropout[0])))
  model.add(layers.TimeDistributed(layers.Dense(fc_size,name='fc2',kernel_initializer=init,bias_initializer=init,activation=cliped_relu)))
  model.add(layers.TimeDistributed(layers.Dropout(dropout[1])))
  model.add(layers.TimeDistributed(layers.Dense(fc_size,name='fc3',kernel_initializer=init,bias_initializer=init,activation=cliped_relu)))
  model.add(layers.TimeDistributed(layers.Dropout(dropout[2])))


  # Layer 4: BiDirectional RNN
  model.add(layers.BiDirectional(layers.LSTM(rnn_size,name='bilstm4',kernel_initializer=keras.initializers.he_normal(),return_sequences=True,activation=cliped_relu, dropout=dropout[3]),merge_mode='sum'))

  # Layer 5: FC
  model.add(layers.TimeDistributed(layers.Dense(fc_size,name='fc5',kernel_initializer=init,bias_initializer=init,activation=cliped_relu)))
  model.add(layers.TimeDistributed(layers.Dropout(dropout[4])))
  # Layer 6: softmax output
  model.add(layers.TimeDistributed(layers.Dense(output_dim,name='out',kernel_initializer=init,bias_initializer=init,activation='softmax')))

  return model
  


In [9]:
my_model=DeepSpeech1()
my_model.summary()

NameError: ignored

Add CTC loss:

In [0]:
def add_ctc(model):
  